In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 500)

types = {
    "Invoice/Item Number": object,
    "Date": object,
    "Store Number": int,
    "Store Name": object,
    "Address": object,
    "City": object,
    "Zip Code": object,
    "Store Location": object,
    "County Number": object,
    "County": object,
    "Category": float,
    "Category Name": object,
    "Vendor Number": float,
    "Vendor Name": object,
    "Item Number": object,
    "Item Description": object,
    "Pack": int,
    "Bottle Volume (ml)": int,
    "State Bottle Cost": float,
    "State Bottle Retail": float,
    "Bottles Sold": int,
    "Sale (Dollars)": float,
    "Volume Sold (Liters)": float,
    "Volume Sold (Gallons)": float,
}

df = pd.read_csv("Iowa_Liquor_Sales.csv", dtype=types, parse_dates=["Date"])
df["Item Number"] = pd.to_numeric(
    df["Item Number"], errors="coerce", downcast="integer"
)
df = df.dropna(subset=["Item Number"])

df_itemNames = df[
    ["Item Number", "Item Description", "Pack", "Bottle Volume (ml)", "Category Name"]
].drop_duplicates(keep="first", subset=["Item Number"]).fillna("-")


df_itemNames["Item Number"] = pd.to_numeric(
    df_itemNames["Item Number"], errors="coerce", downcast="integer"
)

df_storeNames = df[["Store Number","Store Name", "Store Location", "City"]].drop_duplicates(keep="first", subset=["Store Number"])
df_storeNames["Store Number"] = pd.to_numeric(
    df_storeNames["Store Number"], errors="coerce", downcast="integer"
)



In [2]:
df['Store Location'] = df['Store Location'].str[6:]
df['long'] = df['Store Location'].str.split(" ", 2).str[0].str[1:].astype(float)
df['lat'] = df['Store Location'].str.split(" ", 2).str[1].str[:-1].astype(float)

C:\Users\Maxca\AppData\Local\Temp\ipykernel_15728\3484334673.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['long'] = df['Store Location'].str.split(" ", 2).str[0].str[1:].astype(float)
C:\Users\Maxca\AppData\Local\Temp\ipykernel_15728\3484334673.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['lat'] = df['Store Location'].str.split(" ", 2).str[1].str[:-1].astype(float)


In [3]:
locations = df[['long','lat','Volume Sold (Liters)']]\
    .dropna().groupby(['long','lat'])\
        .agg({'Volume Sold (Liters)': 'sum'})\
            .reset_index()\
                .rename(columns={'Volume Sold (Liters)': 'liters'})
                    
locations['coords'] = locations[['long', 'lat']].values.tolist()

# locations[['liters','coords']].to_json("data.json", orient="records")

# print(locations)

In [ ]:
print(len(df))
print(len(df['Item Number'].unique()))
print(len(df['Store Number'].unique()))
print(df['Volume Sold (Liters)'].sum())
print(df['Bottles Sold'].sum())
print(len(df['City'].unique()))
print(df['Sale (Dollars)'].sum())

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:

itemByStorePivot = pd.pivot_table(df, values='Volume Sold (Liters)', index='Item Number', columns='Store Number', aggfunc='sum')

def busiestStore(x):
    
    try:
        return itemByStorePivot[itemByStorePivot.index == x.iloc[0]].idxmax(axis=1).iloc[0].astype(int)
        
    except Exception as e:
        print(x)
        print(e)
        return 0


df['busiestStore'] = df['Item Number']
itemStats = pd.pivot_table(
    df,
    index=['Item Number'],
    aggfunc={'Store Number': lambda x: len(x.unique()),
        'Volume Sold (Liters)':np.sum,
        'State Bottle Retail':np.average,
        'busiestStore': busiestStore,
        }
    ).reset_index()

itemStats.columns = ['item_number', 'price_avg_usd', 'store_count', 'sold_liters', 'busiest_store_id']


itemStats = pd.merge(itemStats, df_storeNames, left_on='busiest_store_id', right_on='Store Number', how='left')\
    .drop(columns=['Store Number', 'Store Location'])\
        .rename(columns={'Store Name':'store_name', 'City':'store_city'})
        
itemStats['store_name'] = itemStats['store_name'].str.split(' / ').str[0]


In [6]:
df['weekday'] = df['Date'].dt.weekday

print(df[['weekday','Item Description']].head())
print(df.weekday.mode())
print(df['weekday'].value_counts())

   weekday               Item Description
0        2      ADMIRAL NELSON SPICED RUM
1        1  JEREMIAH WEED SWEET TEA VODKA
2        2          PATRON SILVER TEQUILA
3        1               FIVE O'CLOCK GIN
4        2        FAMILIA CAMARENA SILVER
0    1
Name: weekday, dtype: int64
1    6239934
2    6178458
0    5802969
3    4904229
4    3477762
5     220280
6      17286
Name: weekday, dtype: int64


In [ ]:
corr = df[['Store Number','Vendor Number','Item Number','Bottle Volume (ml)','State Bottle Retail','Bottles Sold','Volume Sold (Liters)','weekday']].corr()

corr.style.background_gradient(cmap='RdYlGn')

In [7]:
from DataLink import DataLink
import dotenv
import os
dotenv.load_dotenv('../.env')



link = DataLink(os.environ["POSTGRES_USER"], os.environ["POSTGRES_PASSWORD"], "db", 
                host="192.168.0.113", port=6789
                )

# df.set_index('Item Number').to_sql("liquor", link.engine, if_exists="replace", index=True, index_label=['Item Number'], chunksize=1_000_000)
itemStats.set_index('item_number').to_sql("liquor_store_stats", link.engine, if_exists="replace", index=True, index_label=['item_number'], chunksize=1_000_000)


598

In [8]:
pivotDf = (
    pd.pivot_table(
        df,
        index="Store Number",
        columns="Item Number",
        values="Bottles Sold",
        aggfunc=lambda x: x.sum(),
    )
    .reset_index()
    .fillna(0)
    .drop("Store Number", axis=1)
)
print(pivotDf.shape)

(2952, 12598)


In [9]:
from numpy.linalg import svd

matrix = pivotDf.values
u, s, vh = svd(matrix, full_matrices=False)
k = 1024
U = u[:, :k]
VT = vh[:, :k]

In [ ]:
print(pd.DataFrame(VT).head())
print(pd.DataFrame(VT).shape)
print(VT[0].shape)


In [10]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://192.168.0.113:9200")
print(client.info()['tagline'])

You Know, for Search


In [11]:
# 'Item Number','Item Description','Pack','Bottle Volume (ml)','Category Name'

request_body = {
    "mappings": {
        "properties": {
            "productCode": {"type": "integer"},
            "embed": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine",
            },
            "name": {"type": "text"},
            "pack": {"type": "integer"},
            "vol_ml": {"type": "integer"},
            "category": {"type": "text"},
        }
    }
}
print("creating index...")
client.indices.create(index="liquoridx", body=request_body)

creating index...


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'liquoridx'})

In [12]:
for embed, prodCode in zip(VT, pivotDf.columns.to_list()):
    sliceOfMeta = df_itemNames[df_itemNames["Item Number"] == prodCode]
    
    # print(sliceOfMeta)
    
    # break
    doc = {
        "productCode": prodCode,
        "embed": embed,
        "name": sliceOfMeta["Item Description"].iloc[0],
        "pack": sliceOfMeta["Pack"].iloc[0],
        "vol_ml": sliceOfMeta["Bottle Volume (ml)"].iloc[0],
        "category": sliceOfMeta["Category Name"].iloc[0],
    }

    try:
        client.index(index="liquoridx", document=doc)
    except Exception as e:
        print(prodCode, e)

In [13]:
client.indices.refresh(index="liquoridx")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [ ]:
# Fuzzy search

body = {"match": {"name": {"query": "blueberry", "fuzziness": "AUTO"}}}


find = client.search(
    index="liquoridx", query=body, source=["productCode", "name", "embed"]
)
print(find)

for hit in find.body["hits"]["hits"]:
    print(hit)

In [ ]:
# Vector search


# first find the id
searchWord = "Finlandia Rasberry"

body = {"match": {"name": {"query": searchWord, "fuzziness": "AUTO"}}}
find = client.search(index="liquoridx", query=body, source=["productCode", "name"])

topMatchCode = int(find.body["hits"]["hits"][0]["_source"]["productCode"])

# retrieve the embedding for that id

body = {"match": {"productCode": {"query": topMatchCode}}}
find = client.search(
    index="liquoridx", query=body, source=["productCode", "name", "embed"])

embeddingArr = find.body["hits"]["hits"][0]["_source"]["embed"]

# Similarity search


body = {
    "field": "embed",
    "query_vector": embeddingArr,
    "k": 10,
    "num_candidates": 30
}

find = client.search(
    index="liquoridx",
    source=["productCode", "name"],
    knn= body,
)


for hit in find.body["hits"]["hits"]:
    print(hit)